# Node Graph

In [7]:
# Some prerequisites, do not forget to run this!
!pip install ipycytoscape --quiet

In [13]:
# Some more prerequisites, just run this
import networkx as nx
import pandas as pd
import json
from ipycytoscape import *
from pathlib import Path

In [16]:
df_nodes = pd.read_csv(str(Path.cwd() / 'data/data_ariadne_nodes.csv'), sep=';')
df_edges = pd.read_csv(str(Path.cwd() / 'data/data_ariadne_edges.csv'), sep=';')

df_nodes = df_nodes.fillna('')
df_edges = df_edges.fillna('')

df_edges

,id,source,target,label,background-color,shape,line-color,arrow,type,source-arrow-color
0,1,Project Start,Type of Project,,,triangle,#D1E3DD,triangle,triangle,black
1,2,Type of Project,Type of Study,,,triangle,#D1E3DD,triangle,triangle,black
2,3,Type of Study,Data Availability,,,triangle,#D1E3DD,triangle,triangle,black
3,4,Data Availability,Data Simulation,,,,#D1E3DD,,,
4,5,Data Simulation,Experiment Design,,,,#D1E3DD,,,
5,6,Experiment Design,Data Collection,,,,#D1E3DD,,,
6,7,Data Collection,Data Validation,,,,#D1E3DD,,,
7,8,Data Validation,Data Analysis,,,,#D1E3DD,,,
8,9,Data Analysis,Writing,,,,#D1E3DD,,,
9,10,Writing,Dissemination,,,,#D1E3DD,,,


import networkx as nx
import pandas as pd
import json
from ipycytoscape import *


In [10]:
# Import stuff to dynamically update the graph
from ipywidgets import Output
from IPython.display import display

In [11]:
# graph constructor function
def const_graph(nodes_df, edges_df, init):
    # convert df to dicts
    nodes_dict = nodes_df.to_dict('records')
    edges_dict = edges_df.to_dict('records')
    # start building nodes
    data_keys = ['id', 'label'] # cyto logic
    rest_keys = ['score', 'idInt', 'name', 'score', 'group', 'removed', 'selected',
                 'selectable', 'locked', 'grabbed', 'grabbable'] # cyto extra logic
    nodes_graph_list = []
    # now loop over nodes
    for node in nodes_dict:
        dict_node = {}
        data_sub_dict = {'data':{el:node[el] for el in data_keys}} # get MUST data info
        rest_sub_dict = {el:node[el] for el in node.keys() if el in rest_keys} # get extra data info
        dict_node = {**data_sub_dict,**rest_sub_dict} # zip them
        nodes_graph_list.append(dict_node) # add to the list
    # start building edges
    data_keys  = ['id', 'source', 'target'] # cyto logic
    data_keys2 = ['label', 'classes'] # cyto logic
    rest_keys  = ['score', 'weight', 'group', 'networkId', 'networkGroupId', 'intn',
                  'rIntnId', 'group', 'removed', 'selected', 'selectable', 'locked',
                  'grabbed', 'gra bbable', 'classes'] # cyto extra logic
    edges_graph_list = []
    # now loop over edges
    for edge in edges_dict:
        dict_edge = {}
        data_sub_dict = {el:edge[el] for el in data_keys} # get MUST data info
        data_sub_dict2 = {el:edge[el] for el in edge.keys() if el in data_keys2} # get MUST_2 data info
        rest_sub_dict = {el:edge[el] for el in edge.keys() if el in rest_keys} # get extra data info
        dict_edge = {'data':{**data_sub_dict,**data_sub_dict},**rest_sub_dict} # zip them
        edges_graph_list.append(dict_edge) # add to the list
    # create the combined edge+node dictionary
    total_graph_dict = {'nodes': nodes_graph_list, 'edges':edges_graph_list}
    # building the style
    all_node_style = ['background-color', 'background-opacity',
                     'font-family', 'font-size', 'label', 'width',
                     'shape', 'height', 'width', 'text-valign', 'text-halign', 'underlay-color' ,'underlay-shape']
    all_edge_style = ['background-color', 'background-opacity',
                     'font-family', 'font-size', 'label', 'width', 'line-color', 'arrow', 'type', 'target-arrow-shape']
    total_style_dict = {}
    style_elements = []
    # now construct the node styles
    for node in nodes_dict:
        node_dict = {'selector': f'node[id = \"{node["id"]}\"]'}
        style_dict ={"style": { el:node[el] for el in node.keys() if el in all_node_style}}
        node_dict.update(style_dict)
        style_elements.append(node_dict)
    # now construct the edge styles
    for edge in edges_dict:
        edge_dict = {'selector': f'edge[id = \"{edge["id"]}\"]'}
        style_dict = {"style": { el:edge[el] for el in edge.keys() if el in all_edge_style}}
        edge_dict.update(style_dict)
        style_elements.append(edge_dict)
    # now create the graph
    data_graph = json.dumps(total_graph_dict)
    json_to_python = json.loads(data_graph)
    # result_cyto = CytoscapeWidget()
    cyto_graph.graph.clear()
    cyto_graph.graph.add_graph_from_json(json_to_python)
    cyto_graph.set_style(style_elements)
    # also save the initial json to a file
    if init:
        json_filename = 'init_config.json'
        style_filename = 'init_style.json'
        cyto_graph.set_layout(name = 'circle')
    else:
        json_filename = 'curr_config.json'
        style_filename = 'curr_style.json'
        cyto_graph.set_layout(name = 'breadthfirst')
    with open(json_filename, 'w') as outfile:
        json.dump(json_to_python, outfile)
    with open(style_filename, 'w') as outfile:
        json.dump(json_to_python, outfile)
    # and return it
    #return result_cyto

# reset the graph
def res_graph(node):
    with out:
        cyto_graph.graph.clear()
        const_graph(df_nodes[df_nodes['subgraph'] == 'initial'], df_edges[df_nodes['subgraph'] == 'initial'], init=True)

def log_clicks(node):
    with out:
        new_nodes = df_nodes['subgraph'] == node['data']['id'].replace(' ','')
        const_graph(df_nodes[new_nodes], df_edges[new_nodes], init=False)
        if len(df_edges[new_nodes]) == 0 or len(df_nodes[new_nodes]) == 0:
            print('Empty subgraph, resetting view...')
            res_graph(node)
  

# instantiate an jupyternotebook output
out = Output()
# instantiate a graph
cyto_graph = CytoscapeWidget() # ok now create the cytoscape object
const_graph(df_nodes[df_nodes['subgraph'] == 'initial'], df_edges[df_nodes['subgraph'] == 'initial'], init=True)

cyto_graph.on('node', 'click', log_clicks) # dynamically listen to left clicks
cyto_graph.on('node', 'cxttap', res_graph) # dynamically listen to right clicks
cyto_graph.on('node', 'click', log_clicks)


display(cyto_graph) # display the object
display(out) # display the output

CytoscapeWidget(cytoscape_layout={'name': 'circle'}, cytoscape_style=[{'selector': 'node[id = "Project Start"]…

Output()